#  Few-shot распознавание сущностей

Берт и возможность тренировать few-shot ner модели есть в deep pavlov (http://docs.deeppavlov.ai/en/master/features/models/ner.html)

In [1]:
from google.colab import drive
drive.mount('/content/gdrive') #, force_remount=True

Mounted at /content/gdrive


In [2]:
import os
os.chdir('gdrive/My Drive/Colab Notebooks/nlp_hw')

In [3]:
!pip install deeppavlov

     |████████████████████████████████| 993kB 6.7MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 655kB 17.2MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 6.7MB 26.5MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 10.4MB 48.4MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 1.5MB 49.9MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 512kB 43.0MB/s 
     |████████████████████████████████| 2.1MB 45.5MB/s 
     |████████████████████████████████| 860kB 50.3MB/s 
     |████████████████████████████████| 7.3MB 26.2MB/s 
     |████████████████████████████████| 20.1MB 66.5MB/s 
     |████████████████████████████████| 8.2MB 43.1MB/s 
     

In [70]:
!pip install tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 25kB/s 
     |████████████████████████████████| 512kB 40.7MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 3.8MB 44.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=b957ad5cd74b2da928d5d1a8f8de31b9a65c949e968118a68755b62e6aae3223
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninsta

In [71]:
!apt-get --yes install git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.17.1-1ubuntu0.8).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [72]:
!pip install git+https://github.com/deepmipt/bert.git@feat/multi_gpu

  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /tmp/pip-req-build-h3gfdut1
  Running command git clone -q https://github.com/deepmipt/bert.git /tmp/pip-req-build-h3gfdut1
  Created wheel for bert-dp: filename=bert_dp-1.0-cp37-none-any.whl size=23581 sha256=4039a877fba67d92e586e29275f23e8ae051a8d8442224d82dc451a4babc8bb3
  Stored in directory: /tmp/pip-ephem-wheel-cache-bahdg4yf/wheels/1e/41/94/886107eaf932532594886fd8bfc9cb9d4db632e94add49d326
Successfully built bert-dp


In [9]:
pip install razdel

Для удобства достанем предложения подходящего формата регуляркой и раскидаем их на три сета.

In [60]:
with open('avito_cars.txt', 'r', encoding='utf-8') as f:
  texts = f.readlines()

In [64]:
from razdel import tokenize as razdel_tokenize
import re

reSent = re.compile('[Пп]родам.*?\.|[Пп]родаю.*?\.|[Пп]окупаю.*?\.|[Кк]уплю.*?\.', re.DOTALL)

def split_texts(num, file_name, texts):
  sent_count = 0
  f = open(file_name, 'a', encoding='utf-8')
  for t in range(len(texts)):
    sents = re.findall(reSent, texts[t])
    sent_count += len(sents)
    for i in sents:
     words = [token.text for token in list(razdel_tokenize(i)) if token.text != '/']
     for w in words:
      f.write(w+'\n')
    f.write('\n')
    if sent_count == num:
      f.close()
      return t

In [66]:
split_texts(15, 'test.txt', texts[112:])

42

In [67]:
!ls *.txt

avito_cars.txt	test.txt  train.txt  valid.txt


In [69]:
!head test.txt

Продам O
Audi B-AUTO
A I-AUTO
6 I-AUTO
, O
объём O
2 O
. O

Продаю O


Попробуем теперь обучить разметчик сущностей поверх многоязычного берта. Знать, что это такое и как работает не обязательно. Обязательно указать путь к обучающий файлам.

In [73]:
import json
from deeppavlov import configs, build_model, train_model

with configs.ner.ner_ontonotes_bert_mult.open(encoding='utf8') as f:
    ner_config = json.load(f)

ner_config['dataset_reader']['data_path'] = './'  # directory with train.txt, valid.txt and test.txt files
ner_config['metadata']['variables']['NER_PATH'] = './'
ner_config['metadata']['download'] = [ner_config['metadata']['download'][-1]]  # do not download the pretrained ontonotes model

!!! Если колаб крашиться из-за памяти, то после перезапуска не запускайте ячейку с train_model и build_model по очереди. Либо запустите только train_model, либо пропустите ее и переходите к build_model ниже

In [74]:
ner_model = train_model(ner_config, download=True)

2021-04-02 13:00:10.864 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip to /root/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip
100%|██████████| 663M/663M [03:08<00:00, 3.52MB/s]
2021-04-02 13:03:20.946 INFO in 'deeppavlov.core.data.utils'['utils'] at line 268: Extracting /root/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip archive into /root/.deeppavlov/downloads/bert_models
2021-04-02 13:03:28.50 INFO in 'deeppavlov.core.trainers.fit_trainer'['fit_trainer'] at line 68: NNTrainer got additional init parameters ['pytest_max_batches', 'pytest_batch_size'] that will be ignored:


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package perluniprops to /root/nltk_data...


[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.
2021-04-02 13:03:32.519 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /content/gdrive/My Drive/Colab Notebooks/nlp_hw/tag.dict]











The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API




Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use sta

2021-04-02 13:04:06.211 INFO in 'deeppavlov.models.bert.bert_sequence_tagger'['bert_sequence_tagger'] at line 251: [initializing model with Bert from /root/.deeppavlov/downloads/bert_models/multi_cased_L-12_H-768_A-12/bert_model.ckpt]


...

{"valid": {"eval_examples_count": 15, "metrics": {"ner_f1": 72.7273, "ner_token_f1": 91.6667}, "time_spent": "0:14:42", "epochs_done": 29, "batches_seen": 60, "train_examples_seen": 840, "impatience": 0, "patience_limit": 100}}


2021-04-02 13:19:02.487 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/gdrive/My Drive/Colab Notebooks/nlp_hw/tag.dict]
2021-04-02 13:19:50.33 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/gdrive/My Drive/Colab Notebooks/nlp_hw/model]


INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/Colab Notebooks/nlp_hw/model
{"valid": {"eval_examples_count": 15, "metrics": {"ner_f1": 72.7273, "ner_token_f1": 91.6667}, "time_spent": "0:00:06"}}
{"test": {"eval_examples_count": 15, "metrics": {"ner_f1": 69.5652, "ner_token_f1": 91.6667}, "time_spent": "0:00:04"}}


2021-04-02 13:20:28.705 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/gdrive/My Drive/Colab Notebooks/nlp_hw/tag.dict]
2021-04-02 13:20:59.328 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/gdrive/My Drive/Colab Notebooks/nlp_hw/model]


INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/Colab Notebooks/nlp_hw/model


Посмотрим как это все размечается.

In [77]:
ner_model(['ВАЗ 21063', 'Продам ваз 21055', 'Волгу на обмен'])

[[['ВАЗ', '21063'], ['Продам', 'ваз', '21055'], ['Волгу', 'на', 'обмен']],
 [['B-AUTO', 'I-AUTO'], ['O', 'B-AUTO', 'I-AUTO'], ['I-AUTO', 'O', 'O']]]

In [ ]:
marked = []

for text in texts[:300]:
    if len(text.split()) > 100:
        continue
    pred = ner_model([text])
    sent, tags = pred[0][0], pred[1][0]
    
    if len(set(tags[0])) > 1:
        marked.append(list(zip(sent,tags)))

In [81]:
marked[:2]

[[('Ваз', 'B-AUTO'),
  ('2107', 'I-AUTO'),
  ('в', 'O'),
  ('отличном', 'O'),
  ('состоянии', 'O'),
  (',', 'O'),
  ('полностью', 'O'),
  ('на', 'O'),
  ('ходу', 'O'),
  (',', 'O'),
  ('коробка', 'O'),
  ('5ст', 'O'),
  (',', 'O'),
  ('Центральный', 'O'),
  ('замок', 'O'),
  (',', 'O'),
  ('музыка', 'O'),
  (',', 'O'),
  ('хорошая', 'O'),
  ('летняя', 'O'),
  ('резина', 'O'),
  ('.', 'O'),
  ('торг', 'O'),
  ('уместен', 'O'),
  ('.', 'O'),
  ('/', 'O'),
  ('/', 'O'),
  ('Ваз', 'B-AUTO'),
  ('2109', 'I-AUTO'),
  ('1990г', 'O'),
  ('.', 'O'),
  ('в', 'O'),
  ('.', 'O'),
  ('В', 'O'),
  ('отличном', 'O'),
  ('состоянии', 'O'),
  ('на', 'O'),
  ('полном', 'O'),
  ('ходу', 'O'),
  (',', 'O'),
  ('салон', 'O'),
  ('ухоженный', 'O'),
  (',', 'O'),
  ('передние', 'O'),
  ('сиденья', 'O'),
  ('на', 'O'),
  ('пульте', 'O'),
  ('управления', 'O'),
  ('.', 'O'),
  ('Центральный', 'O'),
  ('замок', 'O'),
  (',', 'O'),
  ('музыка', 'O'),
  (',', 'O'),
  ('торг', 'O'),
  ('уместен', 'O'),
  ('.', 'O'

В приницпе марки различаются, однако не в 100% случаях, также некоторые неподходящие слова могут размечаться, как марки.

In [82]:
ner_model = build_model(configs.ner.ner_ontonotes_bert_mult, download=True)

2021-04-02 14:29:14.891 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/deeppavlov_data/ner_ontonotes_bert_mult_v1.tar.gz to /root/.deeppavlov/ner_ontonotes_bert_mult_v1.tar.gz
100%|██████████| 1.32G/1.32G [03:19<00:00, 6.62MB/s]
2021-04-02 14:32:35.296 INFO in 'deeppavlov.core.data.utils'['utils'] at line 268: Extracting /root/.deeppavlov/ner_ontonotes_bert_mult_v1.tar.gz archive into /root/.deeppavlov/models
2021-04-02 14:33:05.862 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip download because of matching hashes
2021-04-02 14:33:06.316 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_ontonotes_bert_mult/tag.dict]
2021-04-02 14:33:39.157 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/ner_ontonotes

INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_ontonotes_bert_mult/model


In [83]:
ner_model(['Передай привет Михаилу Нефедову'])

[[['Передай', 'привет', 'Михаилу', 'Нефедову']],
 [['O', 'O', 'B-PERSON', 'I-PERSON']]]

In [84]:
ner_model(['Поезжай в Москву'])

[[['Поезжай', 'в', 'Москву']], [['O', 'O', 'B-GPE']]]

In [85]:
ner_model(['Поезжай в Альпы'])

[[['Поезжай', 'в', 'Альпы']], [['O', 'O', 'B-LOC']]]

In [86]:
ner_model(['Я говорю на русском и английском языках'])

[[['Я', 'говорю', 'на', 'русском', 'и', 'английском', 'языках']],
 [['O', 'O', 'O', 'B-LANGUAGE', 'O', 'B-LANGUAGE', 'O']]]

In [87]:
ner_model(['Бутылка воды стоит 50 рублей'])

[[['Бутылка', 'воды', 'стоит', '50', 'рублей']],
 [['O', 'O', 'O', 'B-MONEY', 'I-MONEY']]]

In [88]:
ner_model(['ле Биг мак'])

[[['ле', 'Биг', 'мак']], [['B-PERSON', 'I-PERSON', 'I-PERSON']]]

Разметим новые примеры из датасета и посмотрим как справится модель

In [97]:
marked = []

for text in texts[200:400]:
    if len(text.split()) > 100:
        continue
    pred = ner_model([text])
    sent, tags = pred[0][0], pred[1][0]
    
    if len(set(tags[0])) > 1:
        marked.append(list(zip(sent,tags)))

In [102]:
marked[0]

[('KIA', 'B-PRODUCT'),
 ('BONGO', 'I-PRODUCT'),
 ('III', 'I-PRODUCT'),
 (',', 'O'),
 ('2011', 'B-DATE'),
 ('год', 'I-DATE'),
 ('.', 'O'),
 ('4WD', 'B-CARDINAL'),
 ('.', 'O'),
 ('5MКПП', 'B-CARDINAL'),
 ('.', 'O'),
 ('Двухкабинник', 'O'),
 (',', 'O'),
 ('6', 'B-CARDINAL'),
 ('мест', 'O'),
 (',', 'O'),
 ('кондиционер', 'O'),
 (',', 'O'),
 ('противотуманные', 'O'),
 ('фары', 'O'),
 (',', 'O'),
 ('т', 'O'),
 (',', 'O'),
 ('отключаемый', 'O'),
 ('передний', 'O'),
 ('мост', 'O'),
 ('и', 'O'),
 ('пониженная', 'O'),
 ('передача', 'O'),
 ('с', 'O'),
 ('раздатки', 'O'),
 ('4H', 'B-CARDINAL'),
 ('/', 'O'),
 ('4L', 'B-CARDINAL'),
 ('/', 'O'),
 ('2', 'B-CARDINAL'),
 (',', 'O'),
 ('вал', 'O'),
 ('отбора', 'O'),
 ('мощности', 'O'),
 (',', 'O'),
 ('электрические', 'O'),
 ('стеклоподъемники', 'O'),
 (',', 'O'),
 ('подогрев', 'O'),
 ('сидений', 'O'),
 (',', 'O'),
 ('обогрев', 'O'),
 ('зеркал', 'O'),
 (',', 'O'),
 ('электрорегулировка', 'O'),
 ('зеркал', 'O'),
 (',', 'O'),
 ('центральный', 'O'),
 ('замок

Неправильно размеченные примеры:

1) (KIA', 'B-PRODUCT'), ('BONGO', 'I-PRODUCT'), ('III', 'I-PRODUCT'),
 --- тут правильно разметилось начало и продолжение сущности, однако не стоит тег AUTO, которым размечались марки

2) ('4WD', 'B-CARDINAL') ... ('5MКПП', 'B-CARDINAL') --- я не уверена, считаются ли примеры вида "Цифра+буква" за cardinal

3) ('Лада', 'B-ORG'),
  ('Калина', 'I-ORG'),
  ('SPORT', 'I-ORG')

4) ('Авокрепеж', 'B-GPE') --- про магазин

5) ('402', 'B-PRODUCT')

6) ('KIA', 'B-ORG'),
  ('Cerato', 'I-ORG'),
  ('(', 'I-ORG'),
  ('FORTE', 'I-ORG'),
  ('KOUP', 'I-ORG')

7) ('Продам', 'B-PERSON')

Правильно размеченные примеры, которые не были размечены в данных про авто.

1) ('Один', 'B-CARDINAL')

2) ('12', 'B-DATE'),
  ('.', 'I-DATE'),
  ('12', 'I-DATE'),
  ('.', 'I-DATE'),
  ('2010г', 'I-DATE')

3) ('69', 'B-QUANTITY'),
  ('т', 'I-QUANTITY'),

4) ('Карла', 'B-FAC'),
  ('Маркса', 'I-FAC'),
  ('412Б', 'I-FAC') --- про адрес